# Playground for new packages

## HyperOpt
## Catboost
## Sklearn pipelines, pipeline custom transformers
## using seaborn and bokeh for visualization
## catboost for an algo
## use pytest to test .py files
## Use docker, flask, gunicorn to deploy and API
### From there it would be deep learning with Keras and GLOVE for NLP.  You could use isolation forests or seq2seq for anomaly detection

# Importation

In [32]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression
from sklearn.preprocessing import *
from sklearn.linear_model import *
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb

In [3]:
file_pkl = '/Users/charisameeker/Documents/Data/iowa_liquor_clean3.4.cats.pkl'
df = pd.read_pickle(file_pkl)
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)

### Drop columns

In [4]:
df.drop(['date', 'address', 'store_name', 'item_description', 'store_subname', \
        'county', 'category_name', 'vendor_name'], axis=1, inplace=True)

In [6]:
# df.county = df.county.cat.add_categories('nan').fillna('nan')
df.city = df.city.astype('string')
df.zip_code = df.zip_code.astype('int16')

In [7]:
df = df.head(1000)

In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   year                 1000 non-null   int16  
 1   month                1000 non-null   int8   
 2   day                  1000 non-null   int8   
 3   store_number         1000 non-null   int16  
 4   city                 1000 non-null   string 
 5   zip_code             1000 non-null   int16  
 6   county_number        1000 non-null   int8   
 7   category             1000 non-null   int32  
 8   vendor_number        1000 non-null   int16  
 9   item_number          1000 non-null   int32  
 10  pack                 1000 non-null   int16  
 11  bottle_vol           1000 non-null   int64  
 12  bottle_cost          1000 non-null   float32
 13  state_bottle_retail  1000 non-null   float32
 14  bottles_sold         1000 non-null   int16  
 15  sale                 1000 non-null   fl

## We used the below data on Oct 29.

In [8]:
file_pkl = '/Users/charisameeker/Documents/Data/iowa_liquor_clean3.5.cats.1000.pkl'
# df.to_pickle(file_pkl)

# Prepare Data

In [9]:
y = df['fd'].copy().values # fd is the result of applying boxcox on the sale column (fitted data)
X = df.copy().drop(['sale', 'fd'], axis=1)
X_columns = X.copy().columns

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

# Select Model

In [27]:
regression_models = [
    LinearRegression(),
    Ridge(),
    Lasso(),
    ElasticNet(),
    LinearSVR(),
    RandomForestRegressor(),
    GradientBoostingRegressor(),
    xgb.XGBRegressor()
]

for regression_model in regression_models:
    loop_pipe = make_pipeline(preprocessor, regression_model)
    loop_pipe.fit(X_train, y_train)   
    print(f'{regression_model} \n\
    model score: {loop_pipe.score(X_test, y_test):.4f}')

LinearRegression() 
    model score: 0.7501
Ridge() 
    model score: 0.7549
Lasso() 
    model score: -0.0096
ElasticNet() 
    model score: 0.4297
LinearSVR() 
    model score: 0.6108


/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


RandomForestRegressor() 
    model score: 0.9911
GradientBoostingRegressor() 
    model score: 0.9947
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None) 
    model score: 0.9948


# SKLearn Pipeline

In [16]:
df.columns

Index(['year', 'month', 'day', 'store_number', 'city', 'zip_code',
       'county_number', 'category', 'vendor_number', 'item_number', 'pack',
       'bottle_vol', 'bottle_cost', 'state_bottle_retail', 'bottles_sold',
       'sale', 'fd', 'vol_sold', 'store_subnumber'],
      dtype='object')

## RFR

In [67]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate


numeric_features = ["year", "month", "day", 'store_number', 'county_number', \
                    "category", "vendor_number", "item_number", "pack", 'bottle_vol', \
                   'bottle_cost', 'state_bottle_retail', 'bottles_sold',\
                   'vol_sold', 'store_subnumber']
numeric_transformer = [("scaler", StandardScaler())]
numeric_transformer = Pipeline(numeric_transformer)

#pipeline for categorical features
categorical_features = ["city", "zip_code"]
# store_name, item_description, store_subname
categorical_transformer = [("encoder", OneHotEncoder(handle_unknown='ignore'))]
categorical_transformer = Pipeline(categorical_transformer)

#divernce of the pipelines
preprocessor = ColumnTransformer(
    transformers=[("num", numeric_transformer, numeric_features), \
                  ("cat", categorical_transformer, categorical_features)])

#incorporating the random forest estimator
pipeline = make_pipeline(preprocessor, RandomForestRegressor())

# OLD PARAMS
# grid search parameters
params = {'randomforestregressor__n_estimators' : [150, 180, 210, 240],
         'randomforestregressor__max_depth': range(3,7)}

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'randomforestregressor__n_estimators': n_estimators,
               'randomforestregressor__max_features': max_features,
               'randomforestregressor__max_depth': max_depth,
               'randomforestregressor__min_samples_split': min_samples_split,
               'randomforestregressor__min_samples_leaf': min_samples_leaf,
               'randomforestregressor__bootstrap': bootstrap}
print(random_grid)

cv_rf = RandomizedSearchCV(pipeline, random_grid, cv=3)

# 1 pipeline for data transformation
# save as csv
# for loop with sklearn algos, print/plot
#   inspect default params and feature importantance
# decide on model
# 1 pipeline with model and gridsearch

{'randomforestregressor__n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'randomforestregressor__max_features': ['auto', 'sqrt'], 'randomforestregressor__max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'randomforestregressor__min_samples_split': [2, 5, 10], 'randomforestregressor__min_samples_leaf': [1, 2, 4], 'randomforestregressor__bootstrap': [True, False]}


In [68]:
cv_rf.fit(X_train, y_train)

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('scaler',
                                                                                                StandardScaler())]),
                                                                               ['year',
                                                                                'month',
                                                                                'day',
                                                                                'store_number',
                                                                                'county_number',
                                                                                'category',
                                                  

In [70]:
print(f'Best parameters: {cv_rf.best_params_} \n\n\ # max_depth highest value 110
Training accuracy score from tuned model: {cv_rf.best_score_*100:.2f}%')

Best parameters: {'randomforestregressor__n_estimators': 1800, 'randomforestregressor__min_samples_split': 2, 'randomforestregressor__min_samples_leaf': 2, 'randomforestregressor__max_features': 'auto', 'randomforestregressor__max_depth': 110, 'randomforestregressor__bootstrap': True} 

Training accuracy score from tuned model: 98.84%


## Gradient Boosting Regressor

In [90]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate


numeric_features = ["year", "month", "day", 'store_number', 'county_number', \
                    "category", "vendor_number", "item_number", "pack", 'bottle_vol', \
                   'bottle_cost', 'state_bottle_retail', 'bottles_sold',\
                   'vol_sold', 'store_subnumber']
numeric_transformer = [("scaler", StandardScaler())]
numeric_transformer = Pipeline(numeric_transformer)

#pipeline for categorical features
categorical_features = ["city", "zip_code"]
# store_name, item_description, store_subname
categorical_transformer = [("encoder", OneHotEncoder(handle_unknown='ignore'))]
categorical_transformer = Pipeline(categorical_transformer)

#divernce of the pipelines
preprocessor = ColumnTransformer(
    transformers=[("num", numeric_transformer, numeric_features), \
                  ("cat", categorical_transformer, categorical_features)])

#incorporating the random forest estimator
pipeline = make_pipeline(preprocessor, GradientBoostingRegressor())

# OLD PARAMS
# grid search parameters
params = {'GradientBoostingRegressor__max_depth': range(5,17,2),
          'GradientBoostingRegressor__min_samples_split': range(200,1001,200)}
params2 = {'GradientBoostingRegressor__max_features':range(7,20,2)}
params3 = {'gradientboostingregressor__learning_rate': [.1, .3, .5, .7, .9],
          'gradientboostingregressor__n_estimators': [50, 80, 100, 120, 140, 180, 250],
          'gradientboostingregressor__subsample': [.6, .8, 1, 1.2, 1.4, 1.9],
          'gradientboostingregressor__min_samples_split': [1, 2, 3, 4, 6]}

cv_gb = RandomizedSearchCV(pipeline, params3, cv=3)

# 1 pipeline for data transformation
# save as csv
# for loop with sklearn algos, print/plot
#   inspect default params and feature importantance
# decide on model
# 1 pipeline with model and gridsearch

In [91]:
cv_gb.fit(X_train, y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py", line 441, in fit
    self._check_params()
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py", line 251, in _check_params
    raise ValueError("subsample must be in (0,1] but "
ValueError: subsample must be in (0,1] but was 1.4

  warnings.warn("Estimator fit failed. The score on this train-test"
/opt/anaconda3/lib/python3.8

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('scaler',
                                                                                                StandardScaler())]),
                                                                               ['year',
                                                                                'month',
                                                                                'day',
                                                                                'store_number',
                                                                                'county_number',
                                                                                'category',
                                                  

In [94]:
print(f'Best parameters: {cv_gb.best_params_} \n\n\
Training accuracy score from tuned model: {cv_gb.best_score_*100:.2f}%')

Best parameters: {'gradientboostingregressor__subsample': 0.6, 'gradientboostingregressor__n_estimators': 140, 'gradientboostingregressor__min_samples_split': 2, 'gradientboostingregressor__learning_rate': 0.1} 

Training accuracy score from tuned model: 99.66%


##### Parameter Reference

In [107]:
sorted(cv_gb.get_params().keys())

['cv',
 'error_score',
 'estimator',
 'estimator__columntransformer',
 'estimator__columntransformer__cat',
 'estimator__columntransformer__cat__encoder',
 'estimator__columntransformer__cat__encoder__categories',
 'estimator__columntransformer__cat__encoder__drop',
 'estimator__columntransformer__cat__encoder__dtype',
 'estimator__columntransformer__cat__encoder__handle_unknown',
 'estimator__columntransformer__cat__encoder__sparse',
 'estimator__columntransformer__cat__memory',
 'estimator__columntransformer__cat__steps',
 'estimator__columntransformer__cat__verbose',
 'estimator__columntransformer__n_jobs',
 'estimator__columntransformer__num',
 'estimator__columntransformer__num__memory',
 'estimator__columntransformer__num__scaler',
 'estimator__columntransformer__num__scaler__copy',
 'estimator__columntransformer__num__scaler__with_mean',
 'estimator__columntransformer__num__scaler__with_std',
 'estimator__columntransformer__num__steps',
 'estimator__columntransformer__num__verbo

## XGBoost

In [98]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate


numeric_features = ["year", "month", "day", 'store_number', 'county_number', \
                    "category", "vendor_number", "item_number", "pack", 'bottle_vol', \
                   'bottle_cost', 'state_bottle_retail', 'bottles_sold',\
                   'vol_sold', 'store_subnumber']
numeric_transformer = [("scaler", StandardScaler())]
numeric_transformer = Pipeline(numeric_transformer)

#pipeline for categorical features
categorical_features = ["city", "zip_code"]
# store_name, item_description, store_subname
categorical_transformer = [("encoder", OneHotEncoder(handle_unknown='ignore'))]
categorical_transformer = Pipeline(categorical_transformer)

#divernce of the pipelines
preprocessor = ColumnTransformer(
    transformers=[("num", numeric_transformer, numeric_features), \
                  ("cat", categorical_transformer, categorical_features)])

#incorporating the random forest estimator
pipeline = make_pipeline(preprocessor, xgb.XGBRegressor())

# parameters
parameters = {'xgbregressor__nthread':[4], #when use hyperthread, xgboost may become slower
              'xgbregressor__objective':['reg:linear'],
              'xgbregressor__learning_rate': [.03, 0.05, .07], #so called `eta` value
              'xgbregressor__max_depth': [5, 6, 7],
              'xgbregressor__min_child_weight': [4],
              'xgbregressor__silent': [1],
              'xgbregressor__subsample': [0.7],
              'xgbregressor__colsample_bytree': [0.7],
              'xgbregressor__n_estimators': [500]}

# 1 pipeline for data transformation
# save as csv
# for loop with sklearn algos, print/plot
#   inspect default params and feature importantance
# decide on model
# 1 pipeline with model and gridsearch

In [99]:
cv_xgb = RandomizedSearchCV(pipeline, parameters, cv=3)

In [104]:
cv_xgb.fit(X_train, y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:278: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[17:12:45] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1598185652448/work/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[17:12:45] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1598185652448/work/src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:12:46] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1598185652448/work/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[17:12:46] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1598185652448/work/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[17:12:46] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1598185652448/work

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('scaler',
                                                                                                StandardScaler())]),
                                                                               ['year',
                                                                                'month',
                                                                                'day',
                                                                                'store_number',
                                                                                'county_number',
                                                                                'category',
                                                  

In [105]:
print(f'Best parameters: {cv_xgb.best_params_} \n\n\
Training accuracy score from tuned model: {cv_xgb.best_score_*100:.2f}%')

Best parameters: {'xgbregressor__subsample': 0.7, 'xgbregressor__silent': 1, 'xgbregressor__objective': 'reg:linear', 'xgbregressor__nthread': 4, 'xgbregressor__n_estimators': 500, 'xgbregressor__min_child_weight': 4, 'xgbregressor__max_depth': 5, 'xgbregressor__learning_rate': 0.07, 'xgbregressor__colsample_bytree': 0.7} 

Training accuracy score from tuned model: 99.61%


##### Parameter Reference

In [101]:
sorted(cv_xgb.get_params().keys())

['cv',
 'error_score',
 'estimator',
 'estimator__columntransformer',
 'estimator__columntransformer__cat',
 'estimator__columntransformer__cat__encoder',
 'estimator__columntransformer__cat__encoder__categories',
 'estimator__columntransformer__cat__encoder__drop',
 'estimator__columntransformer__cat__encoder__dtype',
 'estimator__columntransformer__cat__encoder__handle_unknown',
 'estimator__columntransformer__cat__encoder__sparse',
 'estimator__columntransformer__cat__memory',
 'estimator__columntransformer__cat__steps',
 'estimator__columntransformer__cat__verbose',
 'estimator__columntransformer__n_jobs',
 'estimator__columntransformer__num',
 'estimator__columntransformer__num__memory',
 'estimator__columntransformer__num__scaler',
 'estimator__columntransformer__num__scaler__copy',
 'estimator__columntransformer__num__scaler__with_mean',
 'estimator__columntransformer__num__scaler__with_std',
 'estimator__columntransformer__num__steps',
 'estimator__columntransformer__num__verbo

# HyperOpt

In [ ]:
from sklearn.metrics import accuracy_score
from hpsklearn import HyperoptEstimator
from hpsklearn import any_regressor
from hpsklearn import any_preprocessing
from hyperopt import tpe

estim = HyperoptEstimator(classifier=any_regressor('my_clf'),
preprocessing=any_preprocessing('my_pre'),
algo=tpe.suggest,
max_evals=3,
trial_timeout=120)

estim.fit(X_train, y_train)

# Show the results
print(f'Score: {estim.score(X_test, y_test)}')